In [1]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
openai.api_type = 'azure'
openai.api_base = "https://husein-ai2-aiservices.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''

In [3]:
def predict(text):
    try:
        messages = [
            {"role": "user", "content": text},
        ]
        response = openai.ChatCompletion.create(
            engine="gpt-35-turbo",
            messages=messages,
            temperature=1.0,
            max_tokens=1024,
            top_p=0.95,
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [4]:
data = []
with open('paragraph-textbooks.jsonl') as fopen:
    for l in fopen:
        data.append({
            'paragraph': json.loads(l)
        })
        
len(data)

49855

In [5]:
data[0]

{'paragraph': '\n\n# Computational Cognitive Science: A Comprehensive Guide":\n\n\n\n## Foreward\n\n\n\nIn the rapidly evolving field of cognitive science, the intersection of computation and cognition has emerged as a critical area of study. This book, "Computational Cognitive Science: A Comprehensive Guide", aims to provide a thorough exploration of this fascinating discipline, offering a deep dive into the theories, methodologies, and applications that define it.\n\n\n\nThe book is structured to provide a comprehensive understanding of the computational approach to cognitive science, starting with the fundamental concepts and gradually progressing to more complex theories and models. It is designed to cater to a wide audience, from students beginning their journey in cognitive science to seasoned researchers seeking a comprehensive reference.\n\n\n\nThe book begins with an exploration of artificial intuition, a concept that has gained significant attention in recent years. It delves

In [6]:
!mkdir question-textbooks

mkdir: cannot create directory ‘question-textbooks’: File exists


In [7]:
def generate(filename, l):
    if os.path.exists(filename):
        return
    
    s = f"""
{l['paragraph']}

anda adalah seorang pelajar yang kurang faham tentang konteks diatas, jana satu soalan dalam bahasa melayu berdasarkan konteks diatas
"""
    s = s.strip()
    r = predict(s)
    l['question'] = r

    with open(filename, 'w') as fopen:
        json.dump(l, fopen)

In [8]:
generate('question-textbooks/0.json', data[0])

In [11]:
from tqdm import tqdm

max_worker = 3
for i in tqdm(range(0, len(data), max_worker)):
    b = data[i: i + max_worker]
    filenames = [(os.path.join('question-textbooks', f'{i + k}.json'), l) for k, l in enumerate(b)]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, *f): f for f in filenames}

        for future in as_completed(futures):
            future.result()

 20%|███████▌                             | 3373/16619 [00:10<01:06, 200.68it/s]

'content'


 27%|██████████                           | 4524/16619 [00:12<00:33, 363.90it/s]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766
The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


 35%|████████████▉                        | 5802/16619 [00:16<00:28, 385.91it/s]

...

 64%|█████████████████████▏           | 10673/16619 [1:11:11<5:49:16,  3.52s/it]

'content'


 74%|████████████████████████▌        | 12376/16619 [3:11:40<4:18:57,  3.66s/it]

'content'


100%|███████████████████████████████████| 16619/16619 [4:24:16<00:00,  1.05it/s]
